1.	Classer les scores en rang (les scores bruts deviennent des rangs classés du plus petit au plus grand)
2.	Transforme les rangs en quantiles normaux (la constante c varie selon la méthode :  Rankit est utilisé ici) On convertit chaque rang en une probabilité cumulative puis on applique l’inverse de la fonction normale standard
3.	Resclaling en scaled score (mean = 10, standard deviation = 3) 


cardsort, flanker, ibam, listsort, oralread, patterncomp, pictvocab

In [8]:
import os

import pandas as pd
import numpy as np
from scipy.stats import norm

In [13]:
repository_path = "c:/Users/Rosalie/OneDrive - Office 365/Documents/UdeS/Hiver 2026/Crédits de recherche/GitHub/FuzzyClustering-PING/"
ping_base_path = "c:/Users/Rosalie/OneDrive - Office 365/Documents/UdeS/Hiver 2026/Crédits de recherche/data/"
output_dir = "c:/Users/Rosalie/OneDrive - Office 365/Documents/UdeS/Hiver 2026/Crédits de recherche/result/inversenormaltransformation/"
os.makedirs(output_dir, exist_ok=True)

In [15]:
# Load all necessary dataframes for cognitive test
dccs = pd.read_csv(f"{ping_base_path}/PINGTabularData/dccs01.txt", sep="\t")
flanker = pd.read_csv(f"{ping_base_path}/PINGTabularData/flanker01.txt", sep="\t")
ibam = pd.read_csv(f"{ping_base_path}/PINGTabularData/ibam01.txt", sep="\t")
lswmt = pd.read_csv(f"{ping_base_path}/PINGTabularData/lswmt01.txt", sep="\t")
orrt = pd.read_csv(f"{ping_base_path}/PINGTabularData/orrt01.txt", sep="\t")
pcps = pd.read_excel(f"{ping_base_path}/PINGAutre/single_pcps.xlsx")
tpvt = pd.read_csv(f"{ping_base_path}/PINGTabularData/tpvt01.txt", sep="\t")

In [16]:
# On crée une copie pour ne pas modifier le dataframe de base
dccsdata = dccs.iloc[1:].copy()
flankerdata = flanker.iloc[1:].copy()
ibamdata = ibam.iloc[1:].copy()
lswmtdata = lswmt.iloc[1:].copy()
orrtdata = orrt.iloc[1:].copy()
pcpsdata = pcps.iloc[1:].copy()
tpvtdata = tpvt.iloc[1:].copy()

# Mettre les colonnes "interview_age" en numérique
dfs = [dccsdata, flankerdata, ibamdata, lswmtdata, orrtdata, pcpsdata, tpvtdata]
for df in dfs:
    df.loc[:, "age_months"] = pd.to_numeric(df["interview_age"], errors="coerce")

# Exclure les participants de moins de 36 mois (3 ans)
dccsdata = dccsdata.loc[dccsdata["age_months"] >= 36].copy()
flankerdata = flankerdata.loc[flankerdata["age_months"] >= 36].copy()
ibamdata = ibamdata.loc[ibamdata["age_months"] >= 36].copy()
lswmtdata = lswmtdata.loc[lswmtdata["age_months"] >= 36].copy()
orrtdata = orrtdata.loc[orrtdata["age_months"] >= 36].copy()
pcpsdata = pcpsdata.loc[pcpsdata["age_months"] >= 36].copy()
tpvtdata = tpvtdata.loc[tpvtdata["age_months"] >= 36].copy()

# Mettre les scores en numérique
dccsdata.loc[:, "cardsort"] = pd.to_numeric(dccsdata["nih_dccs_computed"], errors="coerce")
flankerdata.loc[:, "flanker"] = pd.to_numeric(flankerdata["nih_flanker_computed"], errors="coerce")
ibamdata.loc[:, "ibam"] = pd.to_numeric(ibamdata["tbx_ibam_scr"], errors="coerce")
lswmtdata.loc[:, "listsorting"] = pd.to_numeric(lswmtdata["tbx_ls"], errors="coerce")
orrtdata.loc[:, "oralreading"] = pd.to_numeric(orrtdata["tbx_reading_score"], errors="coerce")
pcpsdata.loc[:, "patterncomp"] = pd.to_numeric(pcpsdata["nih_patterncomp_raw"], errors="coerce")
tpvtdata.loc[:, "pictvocab"] = pd.to_numeric(tpvtdata["tbx_vocab_theta"], errors="coerce")

# Créer les nouveaux dataframes pour les tests avec les colonnes nécessaires
dccs_int = dccsdata.loc[:, ["subjectkey", "age_months", "cardsort"]].iloc[1:].copy()
flanker_int = flankerdata.loc[:, ["subjectkey", "age_months", "flanker"]].iloc[1:].copy()
ibam_int = ibamdata.loc[:, ["subjectkey", "age_months", "ibam"]].iloc[1:].copy()
lswmt_int = lswmtdata.loc[:, ["subjectkey", "age_months", "listsorting"]].iloc[1:].copy()
orrt_int = orrtdata.loc[:, ["subjectkey", "age_months", "oralreading"]].iloc[1:].copy()
pcps_int = pcpsdata.loc[:, ["subjectkey", "age_months", "patterncomp"]].iloc[1:].copy()
tpvt_int = tpvtdata.loc[:, ["subjectkey", "age_months", "pictvocab"]].iloc[1:].copy()

# Étape 1 : classer les scores en rang et création d'une nouvelle colonne "rank"
dccs_int.loc[:, "cardsort_rank"] = dccs_int["cardsort"].rank(method="average")
flanker_int.loc[:, "flanker_rank"] = flanker_int["flanker"].rank(method="average")
ibam_int.loc[:, "ibam_rank"] = ibam_int["ibam"].rank(method="average")
lswmt_int.loc[:, "listsorting_rank"] = lswmt_int["listsorting"].rank(method="average")
orrt_int.loc[:, "oralreading_rank"] = orrt_int["oralreading"].rank(method="average")
pcps_int.loc[:, "patterncomp_rank"] = pcps_int["patterncomp"].rank(method="average")
tpvt_int.loc[:, "pictvocab_rank"] = tpvt_int["pictvocab"].rank(method="average")


In [17]:
# Étape 2 : inverse normal transformation (INT) avec Rankit (c=0.5)

C = 0.5 # Constante de Rankit

# Nombre de valeurs valides (ne compte pas les missings values)
N_dccs = dccs_int["cardsort_rank"].notna().sum()
N_flanker = flanker_int["flanker_rank"].notna().sum()
N_ibam = ibam_int["ibam_rank"].notna().sum()
N_lswmt = lswmt_int["listsorting_rank"].notna().sum()
N_orrt = orrt_int["oralreading_rank"].notna().sum()
N_pcps = pcps_int["patterncomp_rank"].notna().sum()
N_tpvt = tpvt_int["pictvocab_rank"].notna().sum()

# Créer colonne z_score 
# Formule de INT : z = Φ^(-1)((rank - C) / (N + 1 - 2C))
def add_int_zscore(df, rank_col, z_col, C=0.5):
    has_rank = df[rank_col].notna()
    N = has_rank.sum()  # N valide pour CE df
    df[z_col] = np.nan

    df.loc[has_rank, z_col] = norm.ppf(
        (df.loc[has_rank, rank_col] - C) / (N + 1 - 2*C)
    )
    return df

dccs_int    = add_int_zscore(dccs_int,    "cardsort_rank",    "cardsort_z_score")
flanker_int = add_int_zscore(flanker_int, "flanker_rank", "flanker_z_score")
ibam_int    = add_int_zscore(ibam_int,    "ibam_rank",    "ibam_z_score")
lswmt_int   = add_int_zscore(lswmt_int,   "listsorting_rank",   "listsorting_z_score")
orrt_int    = add_int_zscore(orrt_int,    "oralreading_rank",    "oralreading_z_score")
pcps_int    = add_int_zscore(pcps_int,    "patterncomp_rank",    "patterncomp_z_score")
tpvt_int    = add_int_zscore(tpvt_int,    "pictvocab_rank",    "pictvocab_z_score")


In [18]:
# Étape 3 : scaled score avec mean = 10 et SD = 3

dccs_int["cardsort_scaled_score"] = 10 + 3 * dccs_int["cardsort_z_score"]
flanker_int["flanker_scaled_score"] = 10 + 3 * flanker_int["flanker_z_score"]
ibam_int["ibam_scaled_score"] = 10 + 3 * ibam_int["ibam_z_score"]
lswmt_int["listsorting_scaled_score"] = 10 + 3 * lswmt_int["listsorting_z_score"]
orrt_int["oralreading_scaled_score"] = 10 + 3 * orrt_int["oralreading_z_score"]
pcps_int["patterncomp_scaled_score"] = 10 + 3 * pcps_int["patterncomp_z_score"]
tpvt_int["pictvocab_scaled_score"] = 10 + 3 * tpvt_int["pictvocab_z_score"]

print("dccs mean :", dccs_int["cardsort_scaled_score"].mean())
print("dccs SD :", dccs_int["cardsort_scaled_score"].std())

print("flanker mean :", flanker_int["flanker_scaled_score"].mean())
print("flanker SD :", flanker_int["flanker_scaled_score"].std())

print("ibam mean :", ibam_int["ibam_scaled_score"].mean())
print("ibam SD :", ibam_int["ibam_scaled_score"].std())

print("lswmt mean :", lswmt_int["listsorting_scaled_score"].mean())
print("lswmt SD :", lswmt_int["listsorting_scaled_score"].std())

print("orrt mean :", orrt_int["oralreading_scaled_score"].mean())
print("orrt SD :", orrt_int["oralreading_scaled_score"].std())  

print("pcps mean :", pcps_int["patterncomp_scaled_score"].mean())
print("pcps SD :", pcps_int["patterncomp_scaled_score"].std())

print("tpvt mean :", tpvt_int["pictvocab_scaled_score"].mean())
print("tpvt SD :", tpvt_int["pictvocab_scaled_score"].std())

dccs mean : 9.999763473169898
dccs SD : 2.9951677829803014
flanker mean : 10.000433295541407
flanker SD : 2.9981223251880413
ibam mean : 9.991006202686561
ibam SD : 2.9670613619022594
lswmt mean : 9.998450297131473
lswmt SD : 2.976434007844977
orrt mean : 10.001767951336594
orrt SD : 2.992474561640839
pcps mean : 9.999878832641809
pcps SD : 2.996905956693608
tpvt mean : 10.000000000608834
tpvt SD : 2.9996838705324587


In [ ]:
# Export dataframe avec les nouvelles colonnes

dataframes = {
    "dccs_int": dccs_int,
    "flanker_int": flanker_int,
    "ibam_int": ibam_int,
    "lswmt_int": lswmt_int,
    "orrt_int": orrt_int,
    "pcps_int": pcps_int,
    "tpvt_int": tpvt_int,
}

for name, df in dataframes.items():
    file_path = os.path.join(output_dir, f"{name}.xlsx")
    df.to_excel(file_path, index=False)


Line plot with error bars and polynomial fit \
\
Y = Normalized Scaled Scores\
X = Age en année


In [20]:
# Arrondir l'age en année ronde
dfs_int = [dccs_int, flanker_int, ibam_int, lswmt_int, orrt_int, pcps_int, tpvt_int]

for df in dfs_int:
    df["age_year"] = (df["age_months"] / 12).astype(int)


In [ ]:
# Preparer les moyennes par age pour les lineplots de tous les tests
# Grouper par âge Standard Error et Erreur ±2 SE pour tou les tests
def prep_lineplot(df, score_col, age_col="age_year"):
    lineplot_df = (
        df.groupby(age_col)[score_col]
        .agg(["mean", "std", "count"])
        .reset_index()
    )
    lineplot_df["se"] = lineplot_df["std"] / np.sqrt(lineplot_df["count"])
    lineplot_df["error"] = 2 * lineplot_df["se"]
    return lineplot_df

dccs_lineplot = prep_lineplot(dccs_int, "cardsort_scaled_score")
flanker_lineplot = prep_lineplot(flanker_int, "flanker_scaled_score")
ibam_lineplot = prep_lineplot(ibam_int, "ibam_scaled_score")
lswmt_lineplot = prep_lineplot(lswmt_int, "listsorting_scaled_score")
orrt_lineplot = prep_lineplot(orrt_int, "oralreading_scaled_score")
pcps_lineplot = prep_lineplot(pcps_int, "patterncomp_scaled_score")
tpvt_lineplot = prep_lineplot(tpvt_int, "pictvocab_scaled_score")


In [25]:
# Creer line plot avec barres d'erreur et courbe polynomial

import matplotlib.pyplot as plt

# Dictionnaire pour stocker les dataframes de lineplots
lineplots_data = {
    "Card Sort": dccs_lineplot,
    "Flanker": flanker_lineplot,
    "Imitation Based Memory ": ibam_lineplot,
    "List Sorting": lswmt_lineplot,
    "Oral Reading": orrt_lineplot,
    "Pattern Comparison": pcps_lineplot,
    "Picture Vocabulary": tpvt_lineplot
}

# Creer la courbe polynomial avec ajustement polynomial degré 2
def polyfit2(lineplot_df):
    coeffs = np.polyfit(lineplot_df["age_year"], lineplot_df["mean"], 2)
    poly = np.poly1d(coeffs)
    age_range = np.linspace(lineplot_df["age_year"].min(),
                            lineplot_df["age_year"].max(), 200)
    return poly, age_range

polyfit2_results = {name: polyfit2(df_lp) for name, df_lp in lineplots_data.items()}

# Fonction pour créer les line plots avec barres d'erreur et courbe polynomial
def lineplots(lineplot_df, poly,age_range, test_name):

    plt.figure()

    # Points + barres d'erreur
    plt.errorbar(
        lineplot_df["age_year"],
        lineplot_df["mean"],
        yerr=lineplot_df["error"],
        fmt="o",
        capsize=3,
        color="black",
        ecolor="black",
        elinewidth=1,
        markersize=5
    )

    # Ligne reliant les points
    plt.plot(
        lineplot_df["age_year"],
        lineplot_df["mean"],
        color="black",
        linewidth=1.5
    )

    # Courbe polynomial
    plt.plot(
        age_range,
        poly(age_range),
        linestyle="--",
        color="green",
        linewidth=2
    )

    # Titre du graphique et des axes
    plt.title(f"{test_name} Normalized Scaled Score vs Age")
    plt.xlabel("Age (years)")
    plt.ylabel("Normalized Scaled Score")

    # Limites et ticks de l'axe x
    plt.xlim(2, 22)
    plt.xticks(range(3, 22))
  
    file_name = (f"Lineplot_{test_name}_Normalized_Scaled_Score_vs_Age.png")
    full_path = os.path.join(output_dir, file_name)

    plt.savefig(full_path, dpi=300, bbox_inches="tight")

    plt.close() 

# Créer les line plots pour tous les tests
for name, df_lp in lineplots_data.items():
    poly, age_range = polyfit2_results[name]
    lineplots(df_lp, poly, age_range, name.upper())
